# Python-Code paketieren

(Offizielle Dokumentation: [Packaging Python Projects](https://packaging.python.org/tutorials/packaging-projects/))

Wenn wir eine Python-Anwendung mit mehreren Modulen oder auch nur ein einzelnes Skript mit Abhängigkeiten erstellt haben, möchten wir diese in ein Paket zusammenbinden, damit wir diese Funktionalität leicht auf anderen Systemen installieren können.

Beispiel: [surveyresponse.py](surveyresponse.py)

Dieses Paket benötigt unter Linux keine Abhängigkeiten, auf Windows jedoch `pywin32`.

Ein Python-Paket hat üblicherweise folgende Struktur (Beispiel Survey Response-Applikation):
```
survey_response/
├── README.md
├── pyproject.toml
├── survey_response.py
└── test/
    └── test_survey_response.py
```
Im obersten Verzeichnis liegt die Wurzel des (Git-) Repositories. Es enthält ein `README.md` (enthält Verwendungszweck und eine kurze Anleitung), ein Unterverzeichnis mit dem gleichen Namen wie das Projekt sowie das File `pyproject.toml`.

Das Unterverzeichnis enthält das eigentliche Python-Modul. In unserem Beispiel haben wir *kein* Unterverzeichnis, stattdessen liegt der Code im File `survey_response.py` (single file Modul).

Das File `pyproject.toml` ist ein Konfigurationsfile für unser Projekt. Es enthält Projekt-Metadaten, welche zur Paketierung benötigt werden (ausführlichere Anleitung auf der Seite von [setuptools](https://setuptools.pypa.io/en/latest/userguide/declarative_config.html)). Es ersetzt die früher üblichen `setup.cfg` und `setup.py` Files. [Dieser Blogpost](https://ianhopkinson.org.uk/2022/02/understanding-setup-py-setup-cfg-and-pyproject-toml-in-python/) erklärt die etwas komplizierte Geschichte etwas ausführlicher.

**pyproject.toml** ([doc](https://pip.pypa.io/en/stable/reference/build-system/pyproject-toml/))
* globales Konfigurationsfile für das Projekt
* deklariert, welches *build system* verwendet wird (hier: [setuptools](https://setuptools.pypa.io)). Gemeint ist, welches Tool benutzt wird, um unser Projekt zusammenzubauen. Es gibt zwar eine grössere Auswahl, aber setuptools ist das am häufigsten verwendete. Die prominenteste Alternative zu setuptools ist [Poetry](https://python-poetry.org/).
* deklariert Abhängigkeiten, Projekt-Metainformation, Programm-Einstiegspunkte
* enthält Konfiguration von Code Quality Tools wie [pylint](https://pylint.readthedocs.io/en/stable/), [tox](https://tox.wiki/en/4.15.0/), [ruff](https://github.com/astral-sh/ruff) etc.

Hier ist ein Beispiel für eines `pyproject.toml` Konfigurationsfiles für unser Projekt:

```toml
[build-system]
requires = ["setuptools >= 40.6.0", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "survey_response"
version = 0.0.1
authors = [
    {name = "Chuck Norris", email = "python.sysadmins@id.ethz.ch"},
]
description = "Tool to collect mail survey responses in a folder in a csv file."
readme = {file = "README.md", content_type = "text/markdown"}
license = {text = "BSD-3-Clause"}
classifiers = [
    "Programming Language :: Python :: 3",
]
dependencies = [
    "importlib-metadata; platform_system=='Windows'",
]

[options]
zip_safe = False
packages = find:
```

- `name` Der *distribution name* der des Pakets. Falls das Paket auf [pypi.org](pypi.org) veröffentlicht wird, sollte der Name eindeutig sein und noch nicht existieren. 
- `version`: Version des Pakets.
- `description`: Kurze, einzeilige Zusammenfassung des Pakets.
- `readme`: Kann via `file = "README.md"` unser Projekt-README einlesen und als ausführliche Version von `description` dienen.
- `dependencies`: Liste aller importierten Python-Pakete, welche im Paket aufgenommen werden sollen. Diese Liste kann entweder manuell oder mit `find:` automatisiert erzeugt werden.
- `dependencies`: Abhängigkeiten zu externen Python packages. Hier kann auch mit `platform_system=='Windows'` angegeben werden, dass die Abhängigkeit nur unter Windows benötigt wird.

Ist die Paketstruktur mit Code und Setup-Konfiguration bereit, kann das Paket erstellt werden. Zu diesem Zweck muss im Wurzelverzeichnis des Pakets folgender Befehl aufgerufen werden.:

```sh
pip install .
```

Damit ist das Python-Paket lokal installiert. Dies kann mit `pip list` überprüft werden.

Mit folgenden Befehlen kann ein Archiv des Python-Pakets erzeugt werden:

```sh
pip install --upgrade build
python -m build
```
Mit dem ersten Befehl wird sichergestellt, dass die aktuellsten Versionen von *build* installiert ist. Mit dem zweiten Befehl wird das Paket als *tar*-File und als *wheel* verpackt und im Unterverzeichnis *dist* abgelegt.

Dieses File kann nun beispielsweise auf den Python Paket-Index (*PyPI*) [https://pypi.org/](https://pypi.org/) geladen werden. Dieser Upload auf *PyPI* erfolgt am besten mit Hilfe von [twine](https://twine.readthedocs.io/en/latest/).

## Versionsnummern

Die Versionsnummer eines Pakets, welches auf [PyPi](https://pypi.org/) geladen werden soll, muss sich von einem existierenden Paket (mit dem gleichen Namen) unterscheiden. Sinnvollerweies wird die Versionsnummer im Format gemäss
[*Semantic Versioning*](https://semver.org/) angegeben: `MAJOR.MINOR.PATCH`

Vor einem wichtigen Release (z.B. `1.0.0`) müssen unter Umständen sog. Alpha- und Beta-Release veröffentlicht werden. Eine gültie Versionshistorie kann wie folgt aussehen: `0.9.99` -> `1.0.0-alpha.1` -> `1.0.0-alpha.2` -> `1.0.0-beta.1` -> `1.0.0`.

## Kommandozeilen-Tool paketieren

Nachdem man das Paket `ethz-iam-webservice` installiert hat:
```sh
pip install ethz-iam-webservice
```
ist automatisch ein Kommandozeilen-Tool `iam` verfügbar.
Wie macht man so etwas?

Ausführbare Skripte können in [pyproject.toml](https://gitlab.ethz.ch/vermeul/ethz-iam-webservice/-/blob/master/pyproject.toml?ref_type=heads) im Abschnitt `[project.scripts]` wie folgt definiert werden:
```toml
[project.scripts]
iam = ethz_iam_webservice.main:cli
```

[setuptools](https://setuptools.pypa.io) erstellt während der Installation des Paketes zunächst ein kleines, ausführbares «wrapper Skript» namens `iam`. Dieses Skript (unter Unix ein Shell-Skript, unter Windows ein `.exe`) startet den Python und führt schliesslich die Einstiegsmethode `ethz_iam_webservice.main:cli` aus:

```
ethz_iam_webservice.main:cli
      ^              ^    ^
      |              |    |
  module name   main.py  function name
```

```
ethz-iam-webservice/
├── README.md
├── pyproject.toml
├── ethz_iam_webservice/
|   ├── __init__.py
|   ├── main.py
|   └── ...
└── tests/
    ├── test_guests.py
    └── ...
```

Damit dieses «wrapper Skript» bei der Eingabe in der Kommandozeile überhaupt gefunden wird, wird es von `setuptools` in einen der bekannten Ordner (Pfade) platziert, in denen üblicherweise nach Programmen gesucht wird. Die Liste dieser Pfade wird auf allen Plattformen in der Variable `PATH` festgelegt. Einer dieser Pfade ist für lokal installierte Programme vorgesehen:

- `~/.local/bin` (Linux/Apple)
- `AppData\Roaming\bin` (Windows)
- `./venv/bin` (falls ein virtual environment im Einsatz ist)

## Standalone script package

Im Beispiel von `survey_response`, welches nur aus einem `py` File besteht, verwenden wir:

```toml
[project.scripts]
mail_dir2csv = "survey_response:main"
```

```
   survey_response.main
         ^           ^
         |           |
survey_response.py  function name
```

Weiterführende Informationen auf [setuptools.pypa.io](https://setuptools.pypa.io/en/latest/userguide/entry_point.html).

## Beispiel 1: standalone script

```
examples/stand_alone_script/
├── pyproject.toml
└── user.py
```
[`user.py`](examples/stand_alone_script/user.py)
```python
def main():
    print("🎉")
```

[`pyproject.toml`](examples/stand_alone_script/pyproject.toml)
```toml
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "user"
version = "0.0.1"

[project.scripts]
user-ctl = "user:main"
```

## Beispiel 2: multi file package

```
examples/multi_file_package/
├── pyproject.toml  # <- 1 line changed
└── user/
    ├── __init__.py # <- empty
    ├── user.py
    ├── gui.py
    └── cli.py

```
[`user.py`](examples/multi_file_package/user/user.py)
```python
def add_user():
    print("➕👤")
```
[`cli.py`](examples/multi_file_package/user/cli.py)
```python
from user.user import add_user
def main():
    print("CLI")
    add_user()
```
[`gui.py`](examples/multi_file_package/user/gui.py)
```python
from user.user import add_user
def main():
    print("GUI")
    add_user()
```

[`pyproject.toml`](examples/multi_file_package/pyproject.toml)
```diff
 version = "0.0.1"
 
 [project.scripts]
-user-ctl = "user:main"
+user-ctl = "user.cli:main"
```

### Exercise: install your command-line tool locally

- [ ] use `pip install . --prefix local_python_packages` to specify a folder where packages are installed
   - Notice how a `local_python_packages` folder is created
- [ ] add the `local_python_packages` folder to your `PYTHONPATH` (make sure you pick the right Python version!)
  ```sh
  export PYTHONPATH="$PWD/$(find local_python_packages -name site-packages)"
  ```
- [ ] add this path to your `PATH`:
  ```sh
  export PATH="$PWD/local_python_packages/bin:$PATH"
  ```
- [ ] try out the new utility:
  ```sh
  user-ctl
  ```